# Auto Correct

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

## Data Preprocessing

We will use a file called **'shakespeare.txt'** as our corpus and to build our frequency dictionary.

#### Vocabulary

The below steps will be used to preprocess the text file:

- Read the text file
- Convert all the words to lowercase
- Create a list of all the words in the text file
- Create a vocabulary from the list

We will perform the above tasks in the below function of **process_data**.

In [2]:
def process_data(file_name):
    """
    Input: 
        file_name: File (corpus) in current directory to read. 
    Output: 
        words: a list containing all the words in the corpus in lower case. 
    """
    words = []

    with open(file_name) as f:
        text = f.read()                # read the text file
        text_lowercase = text.lower()  # convert text to lowercase
        words = re.findall(r'\w+', text_lowercase)    # create a list of words
        
    f.close()
    
    return words

In [3]:
# create vocabulary
word_l = process_data('shakespeare.txt')
vocab = set(word_l) 

print(f"Length of vocabulary: "+str(len(vocab)))

Length of vocabulary: 6116


#### Word Count Dictionary

Now we will build a word count dictionary where the key is a word and the value is the number of times the word appears in the list. We will implement the **get_count** function to do this.

In [4]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}
    word_count_dict = Counter(word_l)
    
    return word_count_dict

In [5]:
# create the word count dictionary
word_count_dict = get_count(word_l)
print(f"Number of key value pairs in the word count dictionary: "+str(len(word_count_dict)))
print(f"The count for the word 'mine' is {word_count_dict.get('mine',0)}")

Number of key value pairs in the word count dictionary: 6116
The count for the word 'mine' is 116


#### Probabilities of words 

Now we will compute the probability that each word will appear if randomly selected from the corpus of words using the below formula:
$$P(w_i) = \frac{C(w_i)}{M} \$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.


In [6]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}
    
    M = 0
    for word in word_count_dict.keys():
        M += word_count_dict[word]       # calculating total number of words in the corpus
    
    for word in word_count_dict.keys():
        probs[word] = word_count_dict[word] / M    # calculting probability of each word
    
    return probs

In [7]:
probs = get_probs(word_count_dict)

print(f"Number of key value pairs in the probs dictionary: " + str(len(probs)))
print(f"P('mine') is {probs['mine']:.4f}")

Number of key value pairs in the probs dictionary: 6116
P('mine') is 0.0022


## String Manipulations

Below are the functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. We will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**.

In [8]:
def delete_letter(word):
    '''
    Input:
        word: the string/word
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []
    
    # Create a list of 'splits'. This is all the ways a word can be split into Left and Right
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generating all words that result from deleting one character
    delete_l = [L + R[1:] for L, R in split_l if R]

    return delete_l

In [9]:
word = "cans"
delete_word_l = delete_letter(word)
print("Original word: " + word)
print("\nDelete word list:")
print(delete_word_l)

Original word: cans

Delete word list:
['ans', 'cns', 'cas', 'can']


In [10]:
def switch_letter(word):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent character switched
    ''' 
    
    switch_l = []
    split_l = []

    # All the ways a word can be split into Left and Right
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generating all words that result from switching adjacent letters
    switch_l = [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R) > 1]

    return switch_l

In [11]:
word = "cans"
switch_word_l = switch_letter(word)
print("Original word: " + word)
print("\nSwitch word list:")
print(switch_word_l)

Original word: cans

Switch word list:
['acns', 'cnas', 'casn']


In [12]:
def replace_letter(word):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    # All the ways a word can be split into Left and Right
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generating all words that result from replacing each letter with another letter from the alphabet
    replace_l = [L + c + R[1:] for L, R in split_l if R for c in letters]
    
    # getting only unique words from the replace list
    replace_set = set(replace_l)
    
    # removing the original word
    replace_set.discard(word)
    
    # turn the set back into a list and sort it
    replace_l = sorted(list(replace_set))
    
    return replace_l

In [13]:
word = "cans"
replace_word_l = replace_letter(word)
print("Original word: " + word)
print("\nReplace word list:")
print(replace_word_l)

Original word: cans

Replace word list:
['aans', 'bans', 'caas', 'cabs', 'cacs', 'cads', 'caes', 'cafs', 'cags', 'cahs', 'cais', 'cajs', 'caks', 'cals', 'cams', 'cana', 'canb', 'canc', 'cand', 'cane', 'canf', 'cang', 'canh', 'cani', 'canj', 'cank', 'canl', 'canm', 'cann', 'cano', 'canp', 'canq', 'canr', 'cant', 'canu', 'canv', 'canw', 'canx', 'cany', 'canz', 'caos', 'caps', 'caqs', 'cars', 'cass', 'cats', 'caus', 'cavs', 'caws', 'caxs', 'cays', 'cazs', 'cbns', 'ccns', 'cdns', 'cens', 'cfns', 'cgns', 'chns', 'cins', 'cjns', 'ckns', 'clns', 'cmns', 'cnns', 'cons', 'cpns', 'cqns', 'crns', 'csns', 'ctns', 'cuns', 'cvns', 'cwns', 'cxns', 'cyns', 'czns', 'dans', 'eans', 'fans', 'gans', 'hans', 'ians', 'jans', 'kans', 'lans', 'mans', 'nans', 'oans', 'pans', 'qans', 'rans', 'sans', 'tans', 'uans', 'vans', 'wans', 'xans', 'yans', 'zans']


In [14]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    # All the ways a word can be split into Left and Right
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generating all words that result from inserting a letter
    insert_l = [L + c + R[0:] for L, R in split_l for c in letters]

    return insert_l

In [15]:
word = "cans"
insert_word_l = insert_letter(word)
print("Original word: " + word)
print("\nInsert word list:")
print(insert_word_l)

Original word: cans

Insert word list:
['acans', 'bcans', 'ccans', 'dcans', 'ecans', 'fcans', 'gcans', 'hcans', 'icans', 'jcans', 'kcans', 'lcans', 'mcans', 'ncans', 'ocans', 'pcans', 'qcans', 'rcans', 'scans', 'tcans', 'ucans', 'vcans', 'wcans', 'xcans', 'ycans', 'zcans', 'caans', 'cbans', 'ccans', 'cdans', 'ceans', 'cfans', 'cgans', 'chans', 'cians', 'cjans', 'ckans', 'clans', 'cmans', 'cnans', 'coans', 'cpans', 'cqans', 'crans', 'csans', 'ctans', 'cuans', 'cvans', 'cwans', 'cxans', 'cyans', 'czans', 'caans', 'cabns', 'cacns', 'cadns', 'caens', 'cafns', 'cagns', 'cahns', 'cains', 'cajns', 'cakns', 'calns', 'camns', 'canns', 'caons', 'capns', 'caqns', 'carns', 'casns', 'catns', 'cauns', 'cavns', 'cawns', 'caxns', 'cayns', 'cazns', 'canas', 'canbs', 'cancs', 'cands', 'canes', 'canfs', 'cangs', 'canhs', 'canis', 'canjs', 'canks', 'canls', 'canms', 'canns', 'canos', 'canps', 'canqs', 'canrs', 'canss', 'cants', 'canus', 'canvs', 'canws', 'canxs', 'canys', 'canzs', 'cansa', 'cansb', 'cansc

## Combining the edits

#### Edit one letter

The edit_one_letter function gives all the possible edits that are one edit away from a word. The edits consist of the replace, insert, delete, and optionally the switch operation.

In [16]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word
    Output:
        edit_one_set: a set of words with one possible edit
    """
    
    edit_one_set = set()
    
    # getting all the edited words
    delete_l = delete_letter(word)
    insert_l = insert_letter(word)
    replace_l = replace_letter(word)
    switch_l = switch_letter(word)
    
    # concatinating the lists
    if allow_switches == True:
        total_l = delete_l + insert_l + replace_l + switch_l
    else:
        total_l = delete_l + insert_l + replace_l

    # converting into a set to remove duplicates
    edit_one_set = set(total_l)

    return edit_one_set

In [17]:
tmp_word = "my"
tmp_edit_one_set = edit_one_letter(tmp_word)

# turn into list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word: {tmp_word} \n\nList of words one edit away: \n{tmp_edit_one_l}\n")

input word: my 

List of words one edit away: 
['amy', 'ay', 'bmy', 'by', 'cmy', 'cy', 'dmy', 'dy', 'emy', 'ey', 'fmy', 'fy', 'gmy', 'gy', 'hmy', 'hy', 'imy', 'iy', 'jmy', 'jy', 'kmy', 'ky', 'lmy', 'ly', 'm', 'ma', 'may', 'mb', 'mby', 'mc', 'mcy', 'md', 'mdy', 'me', 'mey', 'mf', 'mfy', 'mg', 'mgy', 'mh', 'mhy', 'mi', 'miy', 'mj', 'mjy', 'mk', 'mky', 'ml', 'mly', 'mm', 'mmy', 'mn', 'mny', 'mo', 'moy', 'mp', 'mpy', 'mq', 'mqy', 'mr', 'mry', 'ms', 'msy', 'mt', 'mty', 'mu', 'muy', 'mv', 'mvy', 'mw', 'mwy', 'mx', 'mxy', 'mya', 'myb', 'myc', 'myd', 'mye', 'myf', 'myg', 'myh', 'myi', 'myj', 'myk', 'myl', 'mym', 'myn', 'myo', 'myp', 'myq', 'myr', 'mys', 'myt', 'myu', 'myv', 'myw', 'myx', 'myy', 'myz', 'mz', 'mzy', 'nmy', 'ny', 'omy', 'oy', 'pmy', 'py', 'qmy', 'qy', 'rmy', 'ry', 'smy', 'sy', 'tmy', 'ty', 'umy', 'uy', 'vmy', 'vy', 'wmy', 'wy', 'xmy', 'xy', 'y', 'ym', 'ymy', 'yy', 'zmy', 'zy']



#### Edit two letters

The edit_two_letters function gives all the possible edits that are two edits away from a word. To do so, we will get all the possible edits on a single word and then for each modified word, we will modify it again. The edits consist of the replace, insert, delete, and optionally the switch operation. 

In [18]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    # get all the words one edit away
    edit_one_set = edit_one_letter(word, allow_switches)
    edit_one_l = sorted(list(edit_one_set))
    
    # for each modified word, again get all the words one edit away
    for edit_word in edit_one_l:
        edit_two_word_set = edit_one_letter(edit_word, allow_switches)
        edit_two_set = set.union(edit_two_set, edit_two_word_set)
    
    return edit_two_set

In [19]:
tmp_word = "my"
tmp_edit_two_set = edit_two_letters(tmp_word)

# turn into list to sort it, in order to view it
tmp_edit_two_l = sorted(list(tmp_edit_two_set))

print(f"input word: {tmp_word} \n\nLength of list of words two edits away: {len(tmp_edit_two_l)}\n")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")

input word: my 

Length of list of words two edits away: 7154

First 10 strings ['', 'a', 'aa', 'aamy', 'aay', 'ab', 'abmy', 'aby', 'ac', 'acmy']
Last 10 strings ['zyt', 'zyu', 'zyv', 'zyw', 'zyx', 'zyy', 'zyz', 'zz', 'zzmy', 'zzy']


## Suggest Spelling Suggestions

We will implement `get_suggestions`, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 

**Step 1:** Generate suggestions for a supplied word: 
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use those. 
* Otherwise, suggest the input word.
* The words generated from fewer edits are more likely than words with more edits.

**Step 2**: Create a 'best_words' dictionary where the 'key' is a suggestion and the 'value' is the probability of that word in the vocabulary. If the word is not in the vocabulary, assign it a probability of 0.

**Step 3**: Select the n best suggestions. There may be fewer than n.

In [20]:
def get_suggestions(word, probs, vocab, n=2):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    # get edited words
    edit_one_set = edit_one_letter(word)
    edit_two_set = edit_two_letters(word)
    
    # select the words that are in the dictionary
    sug_word = []
    if word in vocab:
        sug_word.append(word)
    
    sug_edit_one_word = list(set.intersection(edit_one_set, vocab))
    sug_edit_two_word = list(set.intersection(edit_two_set, vocab))
    
    # select the correct list of edited words (step 1 mentioned above)
    suggestions = sug_word or sug_edit_one_word or sug_edit_two_word
    
    # get the probability of the words selected and store in a dictionary
    best_words = {}
    for word in suggestions:
        best_words[word] = probs.get(word,0)
    
    # choose the n best words
    best_words = Counter(best_words)
    n_best = best_words.most_common(n)

    return n_best

## Test the Implementation

In [21]:
def test(word):
    print("Original Word: " + word)
    
    corrections = get_suggestions(word, probs, vocab, 2)
    
    print("\nSuggested Corrections:")
    for i, word_prob in enumerate(corrections):
        print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

In [22]:
test('dys')

Original Word: dys

Suggested Corrections:
word 0: days, probability 0.000410
word 1: dye, probability 0.000019


In [23]:
test('cors')

Original Word: cors

Suggested Corrections:
word 0: cars, probability 0.000019


In [24]:
test('bmacc')

Original Word: bmacc

Suggested Corrections:
word 0: black, probability 0.000354
word 1: back, probability 0.000261
